<a href="https://colab.research.google.com/github/racoope70/daytrading-with-ml/blob/main/lightgbm_enhanced_strategy_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-6-config

In [2]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.1 requires protobuf<6.0.0,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.


In [3]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,801 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,058 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [4]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [5]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [6]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [7]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [8]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0


import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


TensorFlow GPU memory growth enabled


In [9]:
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance --quiet
!pip install stable-baselines3[extra] --quiet


In [10]:
!rm -rf /content/drive

In [12]:
from google.colab import drive
import os
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
#Add Live Updating / Online Learning
!pip install yfinance lightgbm scikit-learn matplotlib


In [14]:
#Noise Filtering
!pip install PyWavelets

In [15]:
pip install -U scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 82.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


In [1]:
!pip install optuna

In [19]:
# === Imports and Warning Fixes ===
import os, gc, time, warnings, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pywt
import optuna
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

warnings.filterwarnings("ignore", message=".*ensure_all_finite.*", category=FutureWarning)

# === Config ===
TEST_MODE = True
TICKERS = ['AAPL', 'TSLA', 'MSFT'] if TEST_MODE else [
    'AAPL', 'TSLA', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'META', 'BRK-B', 'JPM', 'JNJ',
    'XOM', 'V', 'PG', 'UNH', 'MA', 'HD', 'LLY', 'MRK', 'PEP', 'KO', 'BAC', 'ABBV',
    'AVGO', 'COST', 'DIS', 'WMT', 'PFE', 'CSCO', 'TMO', 'ACN', 'CVX', 'ABT', 'CRM',
    'MCD', 'INTC', 'DHR', 'QCOM', 'TXN', 'NEE', 'AMGN', 'LIN', 'PM', 'BMY', 'MDT',
    'HON', 'UNP', 'LOW', 'ORCL', 'NKE', 'RTX', 'MS', 'GS']

PERIOD = "720d"
INTERVAL = "1h"
SAVE_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm/lightgbm_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/QuantConnect/results_lightgbm"
SLIPPAGE_RATE = 0.001
STOP_LOSS_PCT = 0.03
THRESHOLD = 0.55
FEATURES = ["SMA_50", "EMA_20", "RSI_voladj", "MACD_voladj", "Signal_Line", "ATR_voladj", "OBV", "CCI_voladj"]

os.makedirs(SAVE_DIR, exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "plots"), exist_ok=True)
os.makedirs(os.path.join(RESULTS_DIR, "data"), exist_ok=True)

# === Noise Filtering ===
def wavelet_denoise(series, wavelet="db1"):
    coeffs = pywt.wavedec(series, wavelet, mode="symmetric")
    coeffs[1:] = [np.zeros_like(c) for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet, mode="symmetric")[:len(series)]

# === Regime Detection ===
def detect_regime(df, window=100):
    df["Return"] = df["Close"].pct_change()
    df["Vol"] = df["Close"].rolling(window).std()
    valid = df[["Return", "Vol"]].dropna()
    if len(valid) < 50:
        df["Regime"] = 1
        return df
    model = KMeans(n_clusters=3, random_state=42)
    labels = model.fit_predict(valid)
    regime = pd.Series(index=valid.index, data=labels)
    df["Regime"] = regime.reindex(df.index).ffill().bfill()
    return df

# === Feature Engineering ===
def compute_features(df):
    df['SMA_50'] = df['Close'].rolling(50).mean()
    df['EMA_20'] = df['Close'].ewm(span=20).mean()
    delta = df['Close'].diff()
    gain = delta.clip(lower=0).rolling(14).mean()
    loss = -delta.clip(upper=0).rolling(14).mean()
    rs = gain / (loss + 1e-6)
    df['RSI'] = 100 - (100 / (1 + rs))
    df['MACD'] = df['Close'].ewm(12).mean() - df['Close'].ewm(26).mean()
    df['Signal_Line'] = df['MACD'].ewm(9).mean()
    df['ATR'] = df['High'].rolling(14).max() - df['Low'].rolling(14).min()
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    df['CCI'] = (tp - tp.rolling(20).mean()) / (0.015 * tp.rolling(20).std())
    for col in ["RSI", "MACD", "CCI", "OBV", "ATR"]:
        df[f"{col}_voladj"] = wavelet_denoise(df[col].fillna(0).values)
    df = detect_regime(df)
    df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    return df

# === Optuna Tuner ===
def tune_lightgbm_with_optuna(X, y):
    def objective(trial):
        params = {
            "n_estimators": trial.suggest_int("n_estimators", 50, 300),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "num_leaves": trial.suggest_int("num_leaves", 10, 150),
            "random_state": 42,
            "force_row_wise": True,
            "verbosity": -1,
        }
        model = LGBMClassifier(**params)
        model.fit(X_train, y_train)
        probas = model.predict_proba(X_val)[:, 1]
        return roc_auc_score(y_val, probas)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25, show_progress_bar=False)
    return study.best_params

# === Walkforward with Tuning ===
def run_lgbm_walkforward(ticker, window_size=2000, step_size=250, initial_cash=100000):
    print(f"\n{ticker} | LGBM Walkforward")
    df = yf.download(ticker, period=PERIOD, interval=INTERVAL, auto_adjust=False, progress=False)
    if isinstance(df.columns, pd.MultiIndex): df.columns = df.columns.droplevel(1)
    df = compute_features(df)
    if df.empty or not all(col in df.columns for col in FEATURES): return

    scaler = MinMaxScaler()
    capital = initial_cash
    shares = capital / df['Close'].iloc[0]
    port_lgbm, port_hold, y_preds, y_true = [], [], [], []

    for start in range(0, len(df) - window_size, step_size):
        segment = df.iloc[start:start + window_size].copy()
        if segment.shape[0] < window_size: break
        X_all = scaler.fit_transform(segment[FEATURES])
        y_all = segment['Target'].values

        best_params = tune_lightgbm_with_optuna(X_all, y_all)
        model = LGBMClassifier(**best_params, random_state=42, force_row_wise=True)

        X_train, X_test = X_all[:-step_size], X_all[-step_size:]
        y_train, y_test = y_all[:-step_size], y_all[-step_size:]
        prices = segment['Close'].values[-step_size:]

        model.fit(X_train, y_train,
                  eval_set=[(X_test, y_test)],
                  callbacks=[early_stopping(20), log_evaluation(0)])

        probs = model.predict_proba(X_test)[:, 1]
        preds = (probs > THRESHOLD).astype(int)
        y_preds.extend(preds)
        y_true.extend(y_test.tolist())

        for i, pred in enumerate(preds):
            raw_price = prices[i]
            exec_price = raw_price * (1 + SLIPPAGE_RATE) if pred else raw_price * (1 - SLIPPAGE_RATE)
            if pred:
                if raw_price < exec_price * (1 - STOP_LOSS_PCT):
                    capital = shares * raw_price
                else:
                    capital = shares * exec_price
            else:
                shares = capital / exec_price
            port_lgbm.append(capital)
            port_hold.append(shares * raw_price)
        time.sleep(0.01)

    if len(y_preds) < 2: return
    y_preds, y_true = np.array(y_preds), np.array(y_true[:len(y_preds)])
    returns = np.diff(port_lgbm) / (np.array(port_lgbm[:-1]) + 1e-6)
    sharpe = np.mean(returns) / (np.std(returns) + 1e-6) * np.sqrt(252)
    drawdown = np.max(np.maximum.accumulate(port_lgbm) - port_lgbm)

    metrics = {
        "Ticker": ticker,
        "Final_Portfolio": round(port_lgbm[-1], 2),
        "Return_%": round((port_lgbm[-1] - initial_cash) / initial_cash * 100, 2),
        "Sharpe": round(sharpe, 4),
        "Accuracy": round(accuracy_score(y_true, y_preds), 4),
        "Precision": round(precision_score(y_true, y_preds), 4),
        "Recall": round(recall_score(y_true, y_preds), 4),
        "F1_Score": round(f1_score(y_true, y_preds), 4),
        "Drawdown": round(drawdown, 2)
    }

    # Save results
    pd.DataFrame([metrics]).to_csv(f"{RESULTS_DIR}/metrics_{ticker}.csv", index=False)
    model.booster_.save_model(f"{SAVE_DIR}/model_{ticker}.txt")
    with open(os.path.join(RESULTS_DIR, "data", f"scaler_{ticker}.pkl"), "wb") as f:
        pickle.dump(scaler, f)
    with open(os.path.join(RESULTS_DIR, "data", f"features_{ticker}.txt"), "w") as f:
        f.write(",".join(FEATURES))

    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(port_lgbm, label="LGBM Strategy")
    plt.plot(port_hold, label="Buy & Hold")
    plt.title(f"{ticker} Portfolio Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{RESULTS_DIR}/plots/{ticker}_portfolio_plot.png")
    plt.close()
    print(f"{ticker}: Return {metrics['Return_%']}%, Sharpe {metrics['Sharpe']}")

    summary_path = os.path.join(RESULTS_DIR, "lightgbm_walkforward_summary.csv")
    if os.path.exists(summary_path):
        all_df = pd.read_csv(summary_path)
        all_df = all_df[all_df["Ticker"] != ticker]
        all_df = pd.concat([all_df, pd.DataFrame([metrics])], ignore_index=True)
    else:
        all_df = pd.DataFrame([metrics])
    all_df.to_csv(summary_path, index=False)

# === Run All Tickers ===
for ticker in TICKERS:
    run_lgbm_walkforward(ticker)
    gc.collect()
    time.sleep(1)



AAPL | LGBM Walkforward


[I 2025-06-27 03:09:18,302] A new study created in memory with name: no-name-c61bc6cc-716e-4df5-a9cd-7254a0feeaf9
[I 2025-06-27 03:09:18,352] Trial 0 finished with value: 0.554296718687475 and parameters: {'n_estimators': 231, 'learning_rate': 0.29525874959895854, 'max_depth': 4, 'num_leaves': 18}. Best is trial 0 with value: 0.554296718687475.
[I 2025-06-27 03:09:18,391] Trial 1 finished with value: 0.5475565226090436 and parameters: {'n_estimators': 123, 'learning_rate': 0.20167841863979571, 'max_depth': 6, 'num_leaves': 107}. Best is trial 0 with value: 0.554296718687475.
[I 2025-06-27 03:09:18,457] Trial 2 finished with value: 0.5136429571828731 and parameters: {'n_estimators': 171, 'learning_rate': 0.043257984585042125, 'max_depth': 9, 'num_leaves': 73}. Best is trial 0 with value: 0.554296718687475.
[I 2025-06-27 03:09:18,490] Trial 3 finished with value: 0.5012755102040817 and parameters: {'n_estimators': 140, 'learning_rate': 0.07394143786404186, 'max_depth': 4, 'num_leaves': 9

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.694392


[I 2025-06-27 03:09:21,769] Trial 1 finished with value: 0.514473157051282 and parameters: {'n_estimators': 262, 'learning_rate': 0.1254365969479225, 'max_depth': 3, 'num_leaves': 13}. Best is trial 0 with value: 0.5150240384615384.
[I 2025-06-27 03:09:21,898] Trial 2 finished with value: 0.5336288060897436 and parameters: {'n_estimators': 220, 'learning_rate': 0.28234806446274796, 'max_depth': 6, 'num_leaves': 134}. Best is trial 2 with value: 0.5336288060897436.
[I 2025-06-27 03:09:22,013] Trial 3 finished with value: 0.5134715544871795 and parameters: {'n_estimators': 185, 'learning_rate': 0.049152313223302255, 'max_depth': 6, 'num_leaves': 106}. Best is trial 2 with value: 0.5336288060897436.
[I 2025-06-27 03:09:22,067] Trial 4 finished with value: 0.5044946915064104 and parameters: {'n_estimators': 58, 'learning_rate': 0.018630880078405163, 'max_depth': 7, 'num_leaves': 135}. Best is trial 2 with value: 0.5336288060897436.
[I 2025-06-27 03:09:22,169] Trial 5 finished with value: 0

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.70058


...

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.683563


[I 2025-06-27 03:10:37,559] Trial 2 finished with value: 0.5284329927884616 and parameters: {'n_estimators': 223, 'learning_rate': 0.2882741683292466, 'max_depth': 5, 'num_leaves': 117}. Best is trial 2 with value: 0.5284329927884616.
[I 2025-06-27 03:10:37,578] Trial 3 finished with value: 0.4878555689102564 and parameters: {'n_estimators': 58, 'learning_rate': 0.22083170459499518, 'max_depth': 3, 'num_leaves': 78}. Best is trial 2 with value: 0.5284329927884616.
[I 2025-06-27 03:10:37,614] Trial 4 finished with value: 0.5108047876602564 and parameters: {'n_estimators': 63, 'learning_rate': 0.09647937171145472, 'max_depth': 10, 'num_leaves': 88}. Best is trial 2 with value: 0.5284329927884616.
[I 2025-06-27 03:10:37,663] Trial 5 finished with value: 0.5226111778846153 and parameters: {'n_estimators': 149, 'learning_rate': 0.11088505707581474, 'max_depth': 6, 'num_leaves': 117}. Best is trial 2 with value: 0.5284329927884616.
[I 2025-06-27 03:10:37,703] Trial 6 finished with value: 0.5

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.694635


[I 2025-06-27 03:10:39,641] Trial 2 finished with value: 0.546329196276649 and parameters: {'n_estimators': 164, 'learning_rate': 0.014061264205875211, 'max_depth': 8, 'num_leaves': 58}. Best is trial 1 with value: 0.579596636973276.
[I 2025-06-27 03:10:39,687] Trial 3 finished with value: 0.5616429786808127 and parameters: {'n_estimators': 175, 'learning_rate': 0.24919406404090694, 'max_depth': 4, 'num_leaves': 142}. Best is trial 1 with value: 0.579596636973276.
[I 2025-06-27 03:10:39,747] Trial 4 finished with value: 0.5875287758983084 and parameters: {'n_estimators': 123, 'learning_rate': 0.2500913914448735, 'max_depth': 10, 'num_leaves': 37}. Best is trial 4 with value: 0.5875287758983084.
[I 2025-06-27 03:10:39,787] Trial 5 finished with value: 0.5683740366329697 and parameters: {'n_estimators': 77, 'learning_rate': 0.29856957797954203, 'max_depth': 9, 'num_leaves': 99}. Best is trial 4 with value: 0.5875287758983084.
[I 2025-06-27 03:10:39,879] Trial 6 finished with value: 0.575

Training until validation scores don't improve for 20 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.693274
MSFT: Return 6.96%, Sharpe 0.1701
